# Trabajo de Fin de Máster: Applicación Web sobre un modelo de Question Answering en español

    
Descripción de los objetivos del proyecto.
- Imagen de Bert
- ¿Cómo aprenderá Bert?
- ¿Cómo disponibilizaremos la app final?

---
## Capítulo 0: Configuración Inicial

### Importación de librerias

Descripción

In [1]:
import numpy as np
import json
import os
import sys
ABS_DIR = os.path.join(os.getcwd(), "..")
sys.path.append(ABS_DIR)


import utils.read_and_write as rw
import utils.preprocesado as pp
import train.train_utils as tu
import predict.predict_utils as pu

### Configuración del proyecto

Desde la siguiente URL: https://huggingface.co/mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es

* Descargamos los ficheros de entrenamiento y evalución de SQuAD en español
* Descargamos los ficheros de vocab y config con los que se entrenó BETO (Spanish BERT)

In [2]:
CONFIG_DIR = os.path.join(ABS_DIR, "config\\")

DATA_DIR = os.path.join(ABS_DIR, "data\\")
LOG_DIR = os.path.join(DATA_DIR, "logs\\")
MODELS_DIR = os.path.join(DATA_DIR, "models\\")
DATASETS_DIR = os.path.join(DATA_DIR, "datasets\\")
CHECKPOINTS_CALLBACK_DIR = os.path.join(DATA_DIR, "checkpoints\\")
CACHE_DIR = os.path.join(DATA_DIR, "cache\\")

logger = rw.crear_logger("tfm-app.log")

train_path = rw.comprobar_fichero_existe(os.path.join(DATASETS_DIR, "train-v2.0-es.json"), logger)
eval_path = rw.comprobar_fichero_existe(os.path.join(DATASETS_DIR, "dev-v2.0-es.json"), logger)

vocab_BERT_path = rw.comprobar_fichero_existe(os.path.join(MODELS_DIR, "vocab.txt"), logger)
config_BERT_path = rw.comprobar_fichero_existe(os.path.join(MODELS_DIR, "config.json"), logger)

config_file = rw.cargar_config()

2021-10-31 19:31:07,455 tfm-app.log INFO: El fichero log tfm-app.log ha sido creado
2021-10-31 19:31:07,459 tfm-app.log INFO: El fichero train-v2.0-es.json existe en D:\tfm_app\src\project\notebooks\..\data\datasets
2021-10-31 19:31:07,460 tfm-app.log INFO: El fichero dev-v2.0-es.json existe en D:\tfm_app\src\project\notebooks\..\data\datasets
2021-10-31 19:31:07,462 tfm-app.log INFO: El fichero vocab.txt existe en D:\tfm_app\src\project\notebooks\..\data\models
2021-10-31 19:31:07,463 tfm-app.log INFO: El fichero config.json existe en D:\tfm_app\src\project\notebooks\..\data\models


### Visualización de datos

Explicación de cómo son los datos

In [3]:
train_df = rw.json_to_dataframe(train_path)
print(train_df.shape)
eval_df = rw.json_to_dataframe(eval_path)
print(eval_df.shape)

(86818, 6)
(9905, 6)


* ENTRENAMIENTO

In [4]:
id_row = 5000
print(f"Título:\n {train_df.iloc[id_row, 1]}")
print("---"*30)
print(f"Párrafo:\n {train_df.iloc[id_row, 2]}")
print("---"*30)
print(f"Pregunta:\n {train_df.iloc[id_row, 3]}")
print("---"*30)
print(f"Posicion de respuesta:\n {train_df.iloc[id_row, 4]}")
print("---"*30)
print(f"Respuesta:\n {train_df.iloc[id_row, 5]}")

Título:
 Budismo
------------------------------------------------------------------------------------------
Párrafo:
 La segunda verdad es que el origen de la dukkha puede ser conocido. En el contexto de las cuatro verdades nobles, el origen de dukkha se explica comúnmente como deseo (Pali: Tanha) condicionado por la ignorancia (Pali: Avijja). En un nivel más profundo, la causa raíz del dukkha se identifica como ignorancia (Pali: Avijja) de la verdadera naturaleza de las cosas. La tercera verdad noble es que el cese completo de la dukkha es posible, y la cuarta verdad noble identifica un camino a este cese. [Nota 7]
------------------------------------------------------------------------------------------
Pregunta:
 El origen del dukkha se explica como deseo condicionado por qué?
------------------------------------------------------------------------------------------
Posicion de respuesta:
 201
------------------------------------------------------------------------------------------

* VALIDACIÓN

In [5]:
id_row = 5000
print(f"Título:\n {eval_df.iloc[id_row, 1]}")
print("---"*30)
print(f"Párrafo:\n {eval_df.iloc[id_row, 2]}")
print("---"*30)
print(f"Pregunta:\n {eval_df.iloc[id_row, 3]}")
print("---"*30)
print(f"Posicion de respuesta:\n {eval_df.iloc[id_row, 4]}")
print("---"*30)
print(f"Respuesta:\n {eval_df.iloc[id_row, 5]}")

Título:
 Escuela privada
------------------------------------------------------------------------------------------
Párrafo:
 Las escuelas religiosas afiliadas y denominacionales forman una subcategoría de escuelas privadas. Algunas de estas escuelas enseñan educación religiosa, junto con los sujetos académicos habituales para impresionar las creencias y tradiciones de su fe particular en los estudiantes que asisten. Otros usan la denominación como una etiqueta más general para describir lo que los fundadores basaron en su creencia, manteniendo una fina distinción entre académicos y religión. Incluyen escuelas parroquiales, un término que a menudo se utiliza para denotar escuelas católicas romanas. Otros grupos religiosos representados en el sector de la educación privada K-12 incluyen protestantes, judíos, musulmanes y cristianos ortodoxos.
------------------------------------------------------------------------------------------
Pregunta:
 Junto con musulmanes, judíos y cristianos pr

---
## Capítulo 1: Preprocesado de datos

### Configurando el tokenizador

In [6]:
tokenizador = pp.obtener_tokenizador(vocab=vocab_BERT_path, lowercase=False)

In [7]:
tokens = tokenizador.encode("Esto es una prueba para ver cómo se tokeniza")
print(f'ids: {tokens.ids}')
print(f'tokens: {tokens.tokens}')

ids: [4, 2098, 1058, 1108, 3889, 1110, 1292, 2078, 1062, 3, 5]
tokens: ['[CLS]', 'Esto', 'es', 'una', 'prueba', 'para', 'ver', 'cómo', 'se', '[UNK]', '[SEP]']


### Modificando la configuración

In [8]:
with open(config_BERT_path, 'r') as json_file:
    config_BERT = json.load(json_file)
config_BERT.update(config_file["train"]["preprocess"])

### Preprocesamiento

* Entrenamiento

In [9]:
x_train, y_train, train_dataset, train_squad_objects, train_errors = pp.transformar_datos_squad(train_df, tokenizador, config_BERT, logger=logger, name_data="entrenamiento")

2021-10-14 23:27:57,657 tfm-app.log INFO: Transformado el conjunto de entrenamiento al formato SquadExample
2021-10-14 23:28:03,531 tfm-app.log INFO: Se ha conseguido obtener los inputs y los targets para el conjunto de entrenamiento. Se han creado  85629 puntos


* Validación

In [10]:
x_eval, y_eval, eval_dataset, eval_squad_objects, eval_errors = pp.transformar_datos_squad(eval_df, tokenizador, config_BERT, logger=logger, name_data="validación")

2021-10-14 23:28:10,812 tfm-app.log INFO: Transformado el conjunto de validación al formato SquadExample
2021-10-14 23:28:11,388 tfm-app.log INFO: Se ha conseguido obtener los inputs y los targets para el conjunto de validación. Se han creado  9649 puntos


---
## Capítulo 2: Construcción del modelo

### Hiperparámetros

In [9]:
config_BERT

{'attention_probs_dropout_prob': 0.1,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 512,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'type_vocab_size': 2,
 'vocab_size': 31002,
 'max_seq_len': 384,
 'lr': 0.05,
 'batch_size': 64,
 'nb_epoch': 20}

In [10]:
logger.info(f"Modelo entrenado con los siguientes hiperparámetros: {config_BERT}")

2021-10-31 19:31:16,381 tfm-app.log INFO: Modelo entrenado con los siguientes hiperparámetros: {'attention_probs_dropout_prob': 0.1, 'hidden_act': 'gelu', 'hidden_dropout_prob': 0.1, 'hidden_size': 768, 'initializer_range': 0.02, 'intermediate_size': 3072, 'max_position_embeddings': 512, 'num_attention_heads': 12, 'num_hidden_layers': 12, 'type_vocab_size': 2, 'vocab_size': 31002, 'max_seq_len': 384, 'lr': 0.05, 'batch_size': 64, 'nb_epoch': 20}


### Callbacks

In [12]:
callbacks = tu.generar_callbacks(x_eval, y_eval, eval_squad_objects, logger=logger)

### Formato Keras

In [13]:
train_dataset_keras = pp.input_formato_keras(train_dataset, config_BERT)

### Modelo

In [10]:
modelo = tu.obtener_modelo(logger=logger, config=config_BERT)

Some layers from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing TFBertForQuestionAnswering: ['mlm___cls']
- This IS expected if you are initializing TFBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2021-10-17 14:14:52,587 tfm-app.log INFO: Se ha realizado la carga del modelo

In [19]:
modelo.summary()

Model: "tf_bert_for_question_answering"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109850880 
_________________________________________________________________
qa_outputs (Dense)           multiple                  1538      
Total params: 109,852,418
Trainable params: 1,538
Non-trainable params: 109,850,880
_________________________________________________________________


## Capítulo 3: Entrenamiento y validación

### Monitorización en Tensorboard

In [ ]:
# Cargamos el tensorboard
# %load_ext tensorboard
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir '.\..\\data\\logs\\tensorboard\\'

### Entrenamiento

In [21]:
modeloHistory = tu.entrenar_modelo(modelo, train_dataset_keras, config_BERT, callbacks, logger=logger)

2021-10-14 23:30:23,287 tfm-app.log INFO: Se realizará el entrenamiento con 20 epochs


Epoch 1/20
   1/1338 [..............................] - ETA: 0s - loss: 11.8042 - output_1_loss: 5.9276 - output_2_loss: 5.8766WARNING:tensorflow:From d:\python_entornos\tf_gpu\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1338/1338 [==============================] - ETA: 0s - loss: 7.9015 - output_1_loss: 3.9903 - output_2_loss: 3.9112
Epoch 00001: loss improved from inf to 7.90154, saving model to ..\data\checkpoints\model.01-7.90.h5


2021-10-15 01:44:29,703 tfm-app.log INFO: 
epoch=1, exact match score=0.00



epoch=1, exact match score=0.00
1338/1338 [==============================] - 8030s 6s/step - loss: 7.9015 - output_1_loss: 3.9903 - output_2_loss: 3.9112
Epoch 2/20
1338/1338 [==============================] - ETA: 0s - loss: 7.8848 - output_1_loss: 3.9877 - output_2_loss: 3.8971
Epoch 00002: loss improved from 7.90154 to 7.88476, saving model to ..\data\checkpoints\model.02-7.88.h5


2021-10-15 03:58:28,133 tfm-app.log INFO: 
epoch=2, exact match score=0.00


...

2021-10-16 17:55:25,923 tfm-app.log INFO: 
epoch=19, exact match score=0.01



epoch=19, exact match score=0.01
1338/1338 [==============================] - 8027s 6s/step - loss: 7.0027 - output_1_loss: 3.5422 - output_2_loss: 3.4605
Epoch 20/20
1338/1338 [==============================] - ETA: 0s - loss: 7.0103 - output_1_loss: 3.5438 - output_2_loss: 3.4664
Epoch 00020: loss did not improve from 7.00272


2021-10-16 20:09:19,993 tfm-app.log INFO: 
epoch=20, exact match score=0.00



epoch=20, exact match score=0.00
1338/1338 [==============================] - 8028s 6s/step - loss: 7.0103 - output_1_loss: 3.5438 - output_2_loss: 3.4664


2021-10-16 20:09:20,172 tfm-app.log INFO: Entrenamiento finalizado!
2021-10-16 20:09:20,248 tfm-app.log INFO: Guardando pesos...
2021-10-16 20:09:21,484 tfm-app.log INFO: Se han guardado los pesos en: ..\data\models\qa_model_squad_v2_esp\qa_model_squad_v2_esp.h5
2021-10-16 20:09:21,485 tfm-app.log INFO: Guardando Modelo Json...
2021-10-16 20:09:21,487 tfm-app.log ERROR: 


## Capítulo 4: Predicción sobre el modelo entrenado

### Carga del modelo

* Nuestro Modelo

In [11]:
modelo = tu.cargar_modelo(logger=logger, config=config_BERT)

Some layers from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing TFBertForQuestionAnswering: ['mlm___cls']
- This IS expected if you are initializing TFBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2021-10-31 19:31:25,088 tfm-app.log INFO: Se ha cargado el modelo qa_model_sq

In [12]:
modelo.summary()

Model: "tf_bert_for_question_answering"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109850880 
_________________________________________________________________
qa_outputs (Dense)           multiple                  1538      
Total params: 109,852,418
Trainable params: 109,852,418
Non-trainable params: 0
_________________________________________________________________


* Modelo Remoto

In [13]:
modelo_remoto = tu.cargar_modelo(model_name="remote_model", logger=logger)

Some layers from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing TFBertForQuestionAnswering: ['mlm___cls']
- This IS expected if you are initializing TFBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2021-10-31 19:31:28,089 tfm-app.log INFO: Se ha cargado el modelo remote_mode

In [14]:
modelo_remoto.summary()

Model: "tf_bert_for_question_answering_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109850880 
_________________________________________________________________
qa_outputs (Dense)           multiple                  1538      
Total params: 109,852,418
Trainable params: 109,852,418
Non-trainable params: 0
_________________________________________________________________


## Predicción

In [15]:
eval_df.reset_index(drop=True, inplace=True)
print(len(eval_df))

9905


In [27]:
ejemplo_id = 772
context = eval_df.loc[ejemplo_id, "Context"]
print("Contexto:")
print(context, "\n")

question = eval_df.loc[ejemplo_id, "Question"]
print("Pregunta:")
print(question, "\n")

respuesta = eval_df.loc[ejemplo_id, "Text"]
print("Respuesta:")
print(respuesta)

Contexto:
El servicio comenzó el 1 de septiembre de 1993 basado en la idea del entonces director ejecutivo, Sam Chisholm y Rupert Murdoch, de convertir la estrategia de negocio de la compañía a un concepto completamente basado en la fiebre. El nuevo paquete incluye cuatro canales que antes estaban disponibles de forma gratuita, transmitiendo en los satélites de Astra, así como la introducción de nuevos canales. El servicio continuó hasta el cierre del servicio analógico de BSkyB el 27 de septiembre de 2001, debido al lanzamiento y expansión de la plataforma Sky Digital. Algunos de los canales transmitieron ya sea en el cifrado claro o suave (por lo que se requería un decodificador VideoCrypt para decodificar, sin una tarjeta de suscripción) antes de su adición al paquete Sky Multichannels. A los dos meses del lanzamiento, BSkyB ganó 400,000 nuevos suscriptores, con la mayoría tomando al menos un canal premium también, lo que ayudó a BSkyB llegar a 3,5 millones de hogares a mediados de 

* Nuestro Modelo

In [28]:
respuesta_predicha = pu.predict(question, context, modelo=modelo, tokenizer=tokenizador)

In [29]:
respuesta_predicha

{'start': 3,
 'end': 96,
 'answer': 'el 1 de septiembre de 1993 basado en la idea del entonces director ejecutivo, Sam Chisholm y Rupert Murdoch, de convertir la estrategia de negocio de la compañía a un concepto completamente basado en la fiebre. El nuevo paquete incluye cuatro canales que antes estaban disponibles de forma gratuita, transmitiendo en los satélites de Astra, así como la introducción de nuevos canales. El servicio continuó hasta el cierre del servicio analógico de BSkyB el 27 de septiembre de 2001, debido al lanzamiento y expansión de la plataforma Sky Digital. Algunos de los canales transmitieron'}

In [30]:
from IPython.core.display import HTML
HTML(f'<h2>{question.upper()}</h2>')

In [31]:
marked_text = str(context.replace(respuesta_predicha['answer'], f"<mark>{respuesta_predicha['answer']}</mark>"))
HTML(f"""<blockquote> {marked_text} </blockquote>""")

* Modelo Remoto

In [32]:
respuesta_predicha_remoto = pu.predict(question, context, modelo=modelo_remoto, tokenizer=tokenizador)

In [33]:
respuesta_predicha_remoto

{'start': 67, 'end': 68, 'answer': 'continuó hasta'}

In [34]:
from IPython.core.display import HTML
HTML(f'<h2>{question.upper()}</h2>')

In [35]:
marked_text = str(context.replace(respuesta_predicha_remoto['answer'], f"<mark>{respuesta_predicha_remoto['answer']}</mark>"))
HTML(f"""<blockquote> {marked_text} </blockquote>""")

* Usando Pipeline

In [36]:
respuesta_predicha_pipeline = pu.predict(question, context, use_pipeline=True)

In [37]:
respuesta_predicha_pipeline

{'score': 1.104228203985258e-06,
 'start': 341,
 'end': 359,
 'answer': 'satélites de Astra'}

In [40]:
from IPython.core.display import HTML
HTML(f'<h2>{question.upper()}</h2>')

In [41]:
marked_text = str(context.replace(respuesta_predicha_pipeline['answer'], f"<mark>{respuesta_predicha_pipeline['answer']}</mark>"))
HTML(f"""<blockquote> {marked_text} </blockquote>""")

---